# Tidy Data 10/20/22

Goal: Put data into "tidy" (long) format

Timestamp | Temperature | RH | Ambient Temp | Ambient RH  | Room | Window Open 

In [6]:
import IPython

In [7]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel


{'status': 'ok', 'restart': True}

: 

In [1]:
import json
import datetime
import pandas as pd
import numpy as np
import sys
import os
import plotly.express as px

# overall directory location 
root  = "/Users/julietnwagwuume-ezeoke/Library/CloudStorage/GoogleDrive-jnwagwu@stanford.edu/My Drive/UIL/windows/"
# add path to scripts 
sys.path.insert(0, os.path.join(root, 'analysis/scripts'))

In [2]:
from temprh_analysis import Temp_RH_Analysis

# Functions

In [123]:
# TODO add these fx to temprh_analysis.py 

def str2dt(date):
    """Convert a string to a pandas timestamp object """
    return pd.to_datetime(date, format= '%Y, %m, %d, %H, %M' )

def get_htimes(df, date):
    with open('../constants/htimes.json') as f:    
        htimes = json.load(f)
    htimes_arr = [(str2dt(i["open"]), str2dt(i["close"])) for i in  htimes[date]]

    return htimes_arr

def add_window_open_values(df, date):
    htimes = get_htimes(df, date)
    df["Window Open"] = 0 
    # at these values in htime, change to 1 (indicating window open)
    for htime_pair in htimes:
        mask = (df['DateTime'] > htime_pair [0]) & (df['DateTime'] <= htime_pair [1])
        df.loc[mask, "Window Open"] = 1
    return df
        

# Single DataFrame

In [8]:
date = "072522"
a = Temp_RH_Analysis(root, date, ["Open", "Sometimes Open"] )
# TODO make file in constants with all times and variables 

In [36]:
a.trh_422a

,DateTime,Temp C,RH %
0,2022-07-18 22:30:00,26.090,50.099
1,2022-07-18 22:30:30,26.066,50.127
2,2022-07-18 22:31:00,26.042,50.571
3,2022-07-18 22:31:30,26.017,50.185
4,2022-07-18 22:32:00,25.993,50.309
...,...,...,...
18348,2022-07-25 07:24:00,22.901,53.616
18349,2022-07-25 07:24:30,22.925,53.557
18350,2022-07-25 07:25:00,22.901,53.491
18351,2022-07-25 07:25:30,22.925,53.557


* based on this, would just need to add a column about whether the window was open or closed based on h_times, and then group into one data set 

In [13]:
htimes = get_htimes(a, a.date)
htimes

[(Timestamp('2022-07-19 12:01:00'), Timestamp('2022-07-19 20:00:00')),
 (Timestamp('2022-07-20 07:50:00'), Timestamp('2022-07-20 12:30:00')),
 (Timestamp('2022-07-20 22:28:00'), Timestamp('2022-07-21 13:11:00')),
 (Timestamp('2022-07-21 15:06:00'), Timestamp('2022-07-21 17:28:00')),
 (Timestamp('2022-07-21 22:45:00'), Timestamp('2022-07-22 07:50:00')),
 (Timestamp('2022-07-22 11:07:00'), Timestamp('2022-07-22 16:00:00')),
 (Timestamp('2022-07-22 23:02:00'), Timestamp('2022-07-23 19:50:00')),
 (Timestamp('2022-07-24 13:22:00'), Timestamp('2022-07-24 19:33:00'))]

In [ ]:
# create a column window open, with all 0 values (0 indicating closed)
df = a.trh_422a
df["Window Open"] = 0
# at these values in htime, change to 1 (indicating window open)
for htime_pair in htimes:
    mask = (df['DateTime'] > htime_pair [0]) & (df['DateTime'] <= htime_pair [1])
    df.loc[mask, "Window Open"]= 1

df["Window Open"].mean()

In [107]:
try:
    a = 2/0
except:
    print("didnt work")
    pass
a

didnt work


4

## ambient data 

In [119]:
# filtered_df = add_ambient_data(date_data, df, start, end)
def add_ambient_data(date_data, df, start, end):
    """ 
    add ambient data to a pre-filtered dataframe 
    df: dataframe assuming df is filtered according to start and end 
    """
    try:
        date_data.get_ambient_data()
        ad = date_data.ambient_data
    except:
        print(" no ambient data :(")
        return df
        
    # join series into a dataframe and rename
    addf=pd.concat(ad,axis=1)
    addf.rename(columns={"times": "DateTime" , "temps": "Ambient Temp"}, inplace=True)

    # filter to match approp start and end times, reset index 
    f_addf = addf.loc[(addf["DateTime"] >= start)
                    & (addf["DateTime"] <= end)].copy()
    f_addf.reset_index(inplace=True, drop=True)
    f_addf

    # merge and forward fill ambient data 
    mdf = pd.merge_ordered(df, f_addf, on="DateTime", fill_method="ffill")

    # fill in na values at the top of the df w/ the last ambient temp before start of data collection 
    last_pre_cutoff_temp = addf.loc[(addf["DateTime"] <= start)].tail(1)["Ambient Temp"].values[0]
    mdf["Ambient Temp"].fillna(value = last_pre_cutoff_temp, inplace=True)

    # print(f" merged data, heres the proof {mdf.tail()}")

    return mdf


In [14]:
a.get_ambient_data()
ad = a.ambient_data
ad

# hourly intervals -> TODO: apply cutoff times, and add where matching in df? 

{'times': 165   2022-07-18 11:00:00
 164   2022-07-18 12:00:00
 163   2022-07-18 13:00:00
 162   2022-07-18 14:00:00
 161   2022-07-18 15:00:00
               ...        
 4     2022-07-25 04:00:00
 3     2022-07-25 05:00:00
 2     2022-07-25 06:00:00
 1     2022-07-25 07:00:00
 0     2022-07-25 08:00:00
 Name: datetimes, Length: 166, dtype: datetime64[ns],
 'temps': 165    23.444444
 164    24.388889
 163    26.333333
 162    26.555556
 161    26.444444
          ...    
 4      15.000000
 3      15.222222
 2      15.000000
 1      14.888889
 0      15.277778
 Name: Temp F, Length: 166, dtype: float64}

In [9]:
with open('../constants/window_treatment.json') as f:    
    wt_file = json.load(f)

In [12]:
start = str2dt(wt_file[date]["cutoff_times"]["start"])
end = str2dt(wt_file[date]["cutoff_times"]["end"])



# filtered_df = df.loc[(df['DateTime'] >= start)
#                     & (df['DateTime'] <= end)].copy()
# filtered_df.reset_index(inplace=True, drop=True)

In [43]:
addf=pd.concat(ad,axis=1)
addf.rename(columns={"times": "DateTime" , "temps": "Ambient Temp"}, inplace=True)
addf

addf

,DateTime,Ambient Temp
0,2022-07-18 11:00:00,23.444444
1,2022-07-18 12:00:00,24.388889
2,2022-07-18 13:00:00,26.333333
3,2022-07-18 14:00:00,26.555556
4,2022-07-18 15:00:00,26.444444
...,...,...
161,2022-07-25 04:00:00,15.000000
162,2022-07-25 05:00:00,15.222222
163,2022-07-25 06:00:00,15.000000
164,2022-07-25 07:00:00,14.888889


In [59]:
# df = ad["times"]
f_addf = addf.loc[(addf["DateTime"] >= start)
                    & (addf["DateTime"] <= end)].copy()
f_addf.reset_index(inplace=True, drop=True)
f_addf

df = a.trh_422a
f_df = df.loc[(df["DateTime"] >= start)
                    & (df["DateTime"] <= end)].copy()
f_df.reset_index(inplace=True, drop=True)
f_df

,DateTime,Temp C,RH %
0,2022-07-20 07:20:00,23.189,51.268
1,2022-07-20 07:20:30,23.189,51.362
2,2022-07-20 07:21:00,23.189,51.488
3,2022-07-20 07:21:30,23.189,51.457
4,2022-07-20 07:22:00,23.165,51.517
...,...,...,...
11516,2022-07-24 07:18:00,23.285,52.069
11517,2022-07-24 07:18:30,23.285,52.100
11518,2022-07-24 07:19:00,23.309,51.915
11519,2022-07-24 07:19:30,23.309,51.852


In [76]:
last_pre_cutoff_temp = addf.loc[(addf["DateTime"] <= start)].tail(1)["Ambient Temp"]

44    14.333333
Name: Ambient Temp, dtype: float64

In [61]:
f_addf

,DateTime,Ambient Temp
0,2022-07-20 08:00:00,14.666667
1,2022-07-20 09:00:00,15.388889
2,2022-07-20 10:00:00,17.555556
3,2022-07-20 11:00:00,19.777778
4,2022-07-20 12:00:00,19.500000
...,...,...
91,2022-07-24 03:00:00,13.333333
92,2022-07-24 04:00:00,14.277778
93,2022-07-24 05:00:00,14.500000
94,2022-07-24 06:00:00,14.611111


In [56]:
pd.merge(f_df, f_addf, on="DateTime")

,DateTime,Ambient Temp,Temp C,RH %
0,2022-07-20 08:00:00,14.666667,23.477,51.433
1,2022-07-20 09:00:00,15.388889,24.079,50.279
2,2022-07-20 10:00:00,17.555556,24.513,49.605
3,2022-07-20 11:00:00,19.777778,24.827,49.263
4,2022-07-20 12:00:00,19.500000,24.876,49.461
...,...,...,...,...
91,2022-07-24 03:00:00,13.333333,23.477,50.738
92,2022-07-24 04:00:00,14.277778,23.429,50.320
93,2022-07-24 05:00:00,14.500000,23.357,50.595
94,2022-07-24 06:00:00,14.611111,23.309,51.126


In [94]:
mdf = pd.merge_ordered(f_df, f_addf, on="DateTime", fill_method="ffill")
mdf

,DateTime,Temp C,RH %,Ambient Temp
0,2022-07-20 07:20:00,23.189,51.268,NaN
1,2022-07-20 07:20:30,23.189,51.362,NaN
2,2022-07-20 07:21:00,23.189,51.488,NaN
3,2022-07-20 07:21:30,23.189,51.457,NaN
4,2022-07-20 07:22:00,23.165,51.517,NaN
...,...,...,...,...
11516,2022-07-24 07:18:00,23.285,52.069,14.722222
11517,2022-07-24 07:18:30,23.285,52.100,14.722222
11518,2022-07-24 07:19:00,23.309,51.915,14.722222
11519,2022-07-24 07:19:30,23.309,51.852,14.722222


In [95]:
last_pre_cutoff_temp = addf.loc[(addf["DateTime"] <= start)].tail(1)["Ambient Temp"].values[0]
mdf["Ambient Temp"].fillna(value = last_pre_cutoff_temp, inplace=True)
mdf

,DateTime,Temp C,RH %,Ambient Temp
0,2022-07-20 07:20:00,23.189,51.268,14.333333
1,2022-07-20 07:20:30,23.189,51.362,14.333333
2,2022-07-20 07:21:00,23.189,51.488,14.333333
3,2022-07-20 07:21:30,23.189,51.457,14.333333
4,2022-07-20 07:22:00,23.165,51.517,14.333333
...,...,...,...,...
11516,2022-07-24 07:18:00,23.285,52.069,14.722222
11517,2022-07-24 07:18:30,23.285,52.100,14.722222
11518,2022-07-24 07:19:00,23.309,51.915,14.722222
11519,2022-07-24 07:19:30,23.309,51.852,14.722222


In [73]:
mdf.loc[mdf["DateTime"] >= f_addf["DateTime"][0] ]

,DateTime,Temp C,RH %,Ambient Temp
80,2022-07-20 08:00:00,23.477,51.433,14.666667
81,2022-07-20 08:00:30,23.477,51.307,14.666667
82,2022-07-20 08:01:00,23.501,51.373,14.666667
83,2022-07-20 08:01:30,23.501,51.373,14.666667
84,2022-07-20 08:02:00,23.501,51.310,14.666667
...,...,...,...,...
11516,2022-07-24 07:18:00,23.285,52.069,14.722222
11517,2022-07-24 07:18:30,23.285,52.100,14.722222
11518,2022-07-24 07:19:00,23.309,51.915,14.722222
11519,2022-07-24 07:19:30,23.309,51.852,14.722222


# Join All DataFrames

In [251]:
def add_window_open_values(df, date):
    htimes = get_htimes(df, date)
    df["Window Open"] = 0 
    # at these values in htime, change to 1 (indicating window open)
    for htime_pair in htimes:
        mask = (df['DateTime'] > htime_pair [0]) & (df['DateTime'] <= htime_pair [1])
        df.loc[mask, "Window Open"] = 1
    return df

In [151]:
# list of time stamps and experiment data
with open('../constants/window_treatment.json') as f:    
    wt_file = json.load(f)

{'072522': {'422a': 'Open',
  '422b': 'Sometimes Open',
  'cutoff_times': {'start': '2022, 07, 20, 07, 20, 30',
   'end': '2022, 07, 24, 07, 20, 30'}},
 '081622': {'422a': 'Sometimes Closed',
  '422b': 'Closed',
  'cutoff_times': {'start': '2022, 07, 27, 09, 00, 30',
   'end': '2022, 08, 10, 20, 00, 00'}},
 '092022': {'422a': 'Sometimes Closed',
  '422b': 'Closed',
  'cutoff_times': {'start': '2022, 09, 08, 08, 00, 30',
   'end': '2022, 09, 11, 08, 00, 00'}}}

In [124]:
# determine room and window opening closing state for all data
all_data_list = []

for date in wt_file.keys(): # for checking -- list(wt_file.keys())[0:1]
    rooms = [key for key in wt_file[date] if "422" in key]
    for room in rooms:
        date_data = Temp_RH_Analysis(root, date, "_" )
        # print(date_data.ad)

        # assign room values and get appropriate df for the room 
        if room == "422a":
            df =  date_data.trh_422a
            df["Room"] = 0
        elif room == "422b":
            df = date_data.trh_422b
            df["Room"] = 1
        
        # apply cutoff data 
        start = str2dt(wt_file[date]["cutoff_times"]["start"])
        end = str2dt(wt_file[date]["cutoff_times"]["end"])
        filtered_df = df.loc[(df['DateTime'] >= start)
                            & (df['DateTime'] <= end)].copy()
        filtered_df.reset_index(inplace=True, drop=True)

        # insert ambient data on filtered df
        filtered_df = add_ambient_data(date_data, filtered_df, start, end)

        print("-> out of the ambient")
        print(filtered_df.tail(1))


        if wt_file[date][room] == "Open":
            # window open -> 1
            filtered_df["Window Open"] = 1
        elif "Sometimes" in  wt_file[date][room]:
            # window open -> function 
            print("sometimes i work :) ")
            filtered_df = add_window_open_values(filtered_df, date)
        elif wt_file[date][room] == "Closed":
            # window open -> 0
            filtered_df["Window Open"] = 0

        print(filtered_df.tail(1))
        
        # add this data frame to the list 
        all_data_list.append(filtered_df)

# add to total df  
all_data_df = pd.concat(all_data_list, ignore_index=True)
all_data_df

-> out of the ambient
                 DateTime  Temp C    RH %  Room  Ambient Temp
11520 2022-07-24 07:20:00  23.309  51.883     0     14.722222
                 DateTime  Temp C    RH %  Room  Ambient Temp  Window Open
11520 2022-07-24 07:20:00  23.309  51.883     0     14.722222            1
-> out of the ambient
                 DateTime  Temp C    RH %  Room  Ambient Temp
11520 2022-07-24 07:20:00  23.381  53.214     1     14.722222
sometimes i work :) 
                 DateTime  Temp C    RH %  Room  Ambient Temp  Window Open
11520 2022-07-24 07:20:00  23.381  53.214     1     14.722222            0
-> out of the ambient
                 DateTime  Temp C    RH %  Room  Ambient Temp
41640 2022-08-10 20:00:00  25.847  49.587     0     23.333333
sometimes i work :) 
                 DateTime  Temp C    RH %  Room  Ambient Temp  Window Open
41640 2022-08-10 20:00:00  25.847  49.587     0     23.333333            0
-> out of the ambient
                 DateTime  Temp C    RH %  Room 

,DateTime,Temp C,RH %,Room,Ambient Temp,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,1
...,...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,NaN,0
123602,2022-09-11 07:58:30,29.196,50.861,1,NaN,0
123603,2022-09-11 07:59:00,29.196,50.796,1,NaN,0
123604,2022-09-11 07:59:30,29.196,50.796,1,NaN,0


## Checks

In [125]:
all_data_list

[                 DateTime  Temp C    RH %  Room  Ambient Temp  Window Open
 0     2022-07-20 07:20:00  23.189  51.268     0     14.333333            1
 1     2022-07-20 07:20:30  23.189  51.362     0     14.333333            1
 2     2022-07-20 07:21:00  23.189  51.488     0     14.333333            1
 3     2022-07-20 07:21:30  23.189  51.457     0     14.333333            1
 4     2022-07-20 07:22:00  23.165  51.517     0     14.333333            1
 ...                   ...     ...     ...   ...           ...          ...
 11516 2022-07-24 07:18:00  23.285  52.069     0     14.722222            1
 11517 2022-07-24 07:18:30  23.285  52.100     0     14.722222            1
 11518 2022-07-24 07:19:00  23.309  51.915     0     14.722222            1
 11519 2022-07-24 07:19:30  23.309  51.852     0     14.722222            1
 11520 2022-07-24 07:20:00  23.309  51.883     0     14.722222            1
 
 [11521 rows x 6 columns],
                  DateTime  Temp C    RH %  Room  Ambient T

In [126]:
for ix, data in enumerate(all_data_list):
    print(ix, data["Window Open"].mean() )

0 1.0
1 0.5886641784567311
2 0.9482481208424389
3 0.0
4 0.7783821316977202
5 0.0


In [254]:
fig = px.line(all_data_list[4], x="DateTime", y="Window Open" )
fig.show()

In [255]:
all_data_df["Window Open"].mean() 

0.5219406824911412

In [256]:
all_data_df["Room"].mean() # should be 0.5 -> same amount of data collected in Rooms A and B 

0.5

## Save as csv

In [113]:
all_data_df.to_csv("../constants/td_ambient_102022.csv", index=False)

# Weather data